In [1]:
import pandas as pd
import numpy as np

In [8]:
speeches = pd.read_excel("F:\\SynologyDrive\\LSE\\2023\\ST2195 Programming for Data Science\\Week6\\speeches.xlsx")

In [4]:
speeches.head(5)

,date,contents
0,2021-10-20,SPEECH Overcoming the tragedy of the horiz...
1,2021-10-19,SPEECH “Hic sunt leones” – open research q...
2,2021-10-19,SPEECH The role of supervisors and central...
3,2021-10-16,SPEECH Globalisation after the pandemic ...
4,2021-10-14,SPEECH IMFC Statement Statement by Chri...


In [24]:
fx = pd.read_csv("F:\\SynologyDrive\\LSE\\2023\\ST2195 Programming for Data Science\\Week6\\fx.csv", skiprows= 4)

In [25]:
fx.head(5)

,Period\Unit:,[US dollar ]
0,2021-11-16,1.1368
1,2021-11-15,1.1444
2,2021-11-12,1.1448
3,2021-11-11,1.1460
4,2021-11-10,1.1558


In [35]:
fx.dtypes

date         datetime64[ns]
us dollar            object
dtype: object

In [36]:
speeches.dtypes

date        datetime64[ns]
contents            object
dtype: object

In [47]:
# 1. load and merge the datasets keeping all information available for the dates in which there is a measurement in “fx.csv”
fx.rename(columns={"Period\\Unit:":"date", "[US dollar ]":"us dollar"},inplace=True)
fx['date'] = pd.to_datetime(fx['date'])

In [48]:
speeches['date'] = pd.to_datetime(speeches['date'])

In [50]:
#fx_dates_df = fx.join(speeches,how="left", on= "date")
fx_dates_df  = pd.merge(fx, speeches, on ='date', how = 'left')

In [51]:
fx_dates_df

,date,us dollar,contents
0,2021-11-16,1.1368,NaN
1,2021-11-15,1.1444,NaN
2,2021-11-12,1.1448,NaN
3,2021-11-11,1.1460,NaN
4,2021-11-10,1.1558,NaN
...,...,...,...
6502,1999-01-08,1.1659,NaN
6503,1999-01-07,1.1632,NaN
6504,1999-01-06,1.1743,NaN
6505,1999-01-05,1.1790,NaN


In [53]:
#2. Remove entries with obvious outliers or mistakes, if any.
fx_dates_df['date'].value_counts()

2006-05-05    5
2018-05-14    5
2018-03-14    5
2019-03-27    5
2017-04-06    4
             ..
2013-10-28    1
2013-10-29    1
2013-10-30    1
2013-10-31    1
1999-01-04    1
Name: date, Length: 5919, dtype: int64

In [55]:
fx_dates_df.describe()

C:\Users\aowu\AppData\Local\Temp\ipykernel_17884\305582756.py:1: FutureWarning: Treating datetime data as categorical rather than numeric in `.describe` is deprecated and will be removed in a future version of pandas. Specify `datetime_is_numeric=True` to silence this warning and adopt the future behavior now.
  fx_dates_df.describe()


,date,us dollar,contents
count,6507,6507,2323
unique,5919,3588,2323
top,2006-05-05 00:00:00,-,SPEECH Overcoming the tragedy of the horiz...
freq,5,62,1
first,1999-01-04 00:00:00,NaN,NaN
last,2021-11-16 00:00:00,NaN,NaN


In [57]:
#3. Handle missing observations for the exchange rate, if any. This should be done replacing any missing exchange rate with the latest information available. Whenever this cannot be done, the relevant entry should be removed entirely from the dataset.
fx_dates_df['us dollar'].fillna(method='bfill', inplace=True)

In [58]:
fx_dates_df.isna().sum()

date            0
us dollar       0
contents     4184
dtype: int64

In [60]:
#4. Calculate the exchange rate return. Extend the original dataset with the following variables: “good_news” (equal to 1 when the exchange rate return is larger than 0.5 percent, 0 otherwise) and “bad_news” (equal to 1 when the exchange rate return is lower than -0.5 percent, 0 otherwise)
fx_dates_df['return'] = fx_dates_df['us dollar'].diff(-1)/fx_dates_df['us dollar']

TypeError: unsupported operand type(s) for -: 'str' and 'str'

In [62]:
fx_dates_df['us dollar'] = pd.to_numeric(fx_dates_df['us dollar'])

ValueError: Unable to parse string "-" at position 2709

In [ ]:
fx[fx['exchange_rate'].isna()]

In [63]:
fx_dates_df['us dollar'].isna().sum()

0

In [ ]:
# 5. Remove the entries for which contents column has NA values. Generate and store in csv the following tables
# a. “good_indicators” – with the 20 most common words (excluding articles, prepositions and similar connectors) associated with entries wherein “good_news” is equal to 1;
# b. “bad_indicators” – with the 20 most common words (excluding articles, prepositions and similar connectors) associated with entries wherein “bad_news” is equal to 1